In [ ]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from konlpy.tag import Okt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm.auto import tqdm

In [ ]:
train_data_path = "../../data/garments_train.csv"
test_data_path = "../../data/garments_test.csv"

In [ ]:
X_col, y_col = "SentimentText", "Aspect"

In [ ]:
train_df = pd.read_csv(train_data_path).loc[:, [X_col, y_col]].drop_duplicates().reset_index(drop=True)
train_df.head(2)

,SentimentText,Aspect
0,사이즈가잘맞네요,사이즈
1,좀크게나온듯,사이즈


In [ ]:
label_encoder = LabelEncoder()
enc_data = label_encoder.fit_transform(train_df[y_col])
num_labels = len(set(enc_data))

In [ ]:
label_items = label_encoder.classes_
label_numbers = label_encoder.transform(label_items)
dict(zip(label_items, label_numbers))

{'가격': 0, '기능': 1, '디자인': 2, '사이즈': 3, '품질': 4}

In [ ]:
X_train, y_train = train_df.loc[:, X_col].to_list(), enc_data

In [ ]:
okt = Okt()

def discompose(text):
    result = []
    text = re.sub("[^가-힣]", "", text)
    morph_list = okt.morphs(text, norm=True, stem=True)
    stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
    for morph in morph_list:
        if morph not in stopwords:
            result.append(morph)
    return result

In [ ]:
X_train_pos = []
for x in tqdm(X_train):
  X_train_pos.append(discompose(x))
X_train_pos[:2]

  0%|          | 0/45024 [00:00<?, ?it/s]

[['사이즈', '잘맞다'], ['좀', '크게', '나오다']]

In [ ]:
MAX_LEN = 27

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train_pos)

def encode(x):
    sequence = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequence, maxlen=MAX_LEN, padding="post")

X_train_encoding = encode(X_train_pos)
X_train_encoding[:5]

array([[  7,  50,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [ 21,  56,  45,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [ 14,   2,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  5,   6,  86,   6,  13,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  2,  18,  12, 127,  87,  10,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]], dtype=int32)

In [ ]:
model = Sequential([
    Embedding(20000, 300, input_length=MAX_LEN),
    LSTM(units=50),
    Dense(num_labels, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 300)           6000000   
                                                                 
 lstm (LSTM)                 (None, 50)                70200     
                                                                 
 dense (Dense)               (None, 5)                 255       
                                                                 
Total params: 6,070,455
Trainable params: 6,070,455
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_encoding, y_train, epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
1267/1267 [==============================] - 150s 115ms/step - loss: 0.7958 - accuracy: 0.6910 - val_loss: 0.3213 - val_accuracy: 0.9085
Epoch 2/3
1267/1267 [==============================] - 141s 111ms/step - loss: 0.2078 - accuracy: 0.9389 - val_loss: 0.2072 - val_accuracy: 0.9336
Epoch 3/3
1267/1267 [==============================] - 148s 117ms/step - loss: 0.1198 - accuracy: 0.9650 - val_loss: 0.2167 - val_accuracy: 0.9358


In [ ]:
X_col = "RawText"

In [ ]:
test_df = pd.read_csv(test_data_path).loc[:, [X_col, y_col]].drop_duplicates().reset_index(drop=True)
test_df.head(2)

,RawText,Aspect
0,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,품질
1,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,기능


In [ ]:
def test(x):
    labels = ["디자인", "사이즈", "가격", "품질", "기능"]
    aspects = x["Aspect"].to_list()
    result = []
    for label in labels:
        if label in aspects:
            result.append(1)
        else:
            result.append(0)
    return np.array(result)

onehot_df = test_df.groupby(X_col).apply(test).reset_index().rename(columns={ 0: "LabelList" })
onehot_df.head(2)

,RawText,LabelList
0,(12/25)생각 이상으로 아주 좋습니다. 한번 세탁해서 착용하라는 스티커대로 세탁...,"[1, 0, 0, 1, 0]"
1,*****최악최악최악최악최악***** 엄마가 선물 받아 기분 좋다고 바로 신고 나가...,"[0, 0, 0, 1, 0]"


In [ ]:
X_test = onehot_df.loc[:, X_col].to_list()
X_test[:2]

['(12/25)생각 이상으로 아주 좋습니다. 한번 세탁해서 착용하라는 스티커대로 세탁해서 입죠. 디자인 좋고 니트 짜임새 느낌 좋네요. 굿 입니다. (12/28) 세탁 했는데도 많이 묻어납니다.  두번째로 세탁하고 있습니다. 건조 후 확인하고 내용 업데이트하죠 (12/29) 2번 세탁해도 기모털이 많이 묻어나서 못 입겠다는 결론... 반품 바랍니다',
 '*****최악최악최악최악최악***** 엄마가 선물 받아 기분 좋다고 바로 신고 나가심 외부착화는 당연히 잘못된거 알지만 좌우가 눈에 보이게 한쪽이 헐떡헐떡 벗겨져서 고객센터 연락했더니 자로재서 숫자 알려달라함 알려줬더니 사진 찍어 보내라함 이번엔 자로잰 사진으로는 확인이 안된다며 두짝을 겹쳐 찍으라 세번째 요청이 옴 원래 짝발도 아니고 한쪽이 그렇게 헐떡거리면 분명 제품의 품질 문제라고 생각됨 직통번호도 없어서 두번이나 연락처 남기고 전화 기다렸는데 안오고 내가 말하는데 상담원이 이중으로 말해서 더 기분이 안좋아짐 선물하고 기분더러움']

In [ ]:
X_test_pos = []
for x in tqdm(X_test):
  X_test_pos.append(discompose(x))
X_test_pos[:2]

  0%|          | 0/7073 [00:00<?, ?it/s]

[['생각',
  '이상',
  '으로',
  '아주',
  '좋다',
  '한번',
  '세탁',
  '하다',
  '착용',
  '라는',
  '스티커',
  '대',
  '로세',
  '탁하다',
  '입다',
  '디자인',
  '좋다',
  '니트',
  '짜임새',
  '느낌',
  '좋다',
  '굿',
  '이다',
  '탁하다',
  '많이',
  '묻다',
  '나다',
  '두번째',
  '로세',
  '탁하다',
  '건조',
  '후',
  '확인',
  '하고',
  '내용',
  '업데이트',
  '죠',
  '번',
  '세탁',
  '해도',
  '기모',
  '털',
  '많이',
  '묻다',
  '나서다',
  '입다',
  '결론',
  '반품',
  '바라다'],
 ['최악',
  '최악',
  '엄마',
  '선물',
  '받다',
  '기분',
  '좋다',
  '바로',
  '신고',
  '나',
  '가시다',
  '외부',
  '착',
  '화',
  '당연하다',
  '잘못',
  '된거',
  '알',
  '지만',
  '좌우',
  '가누다',
  '보이',
  '게',
  '한쪽',
  '헐떡헐떡',
  '벗겨지다',
  '고객',
  '센터',
  '연락',
  '하다',
  '자로',
  '재다',
  '숫자',
  '알다',
  '달라',
  '함',
  '알다',
  '주다',
  '사진',
  '찍다',
  '보내다',
  '함',
  '이번',
  '엔',
  '자로',
  '재다',
  '사진',
  '으로는',
  '확인',
  '안되다',
  '두',
  '짝',
  '을',
  '겹',
  '치다',
  '찍다',
  '세번',
  '째',
  '요청',
  '옴',
  '원',
  '래',
  '짝',
  '발도',
  '아니다',
  '한쪽',
  '그렇게',
  '헐떡거리다',
  '분명',
  '제품',
  '품질',
  '문제',
  '라고',
  '생각',
  '되다',
  '

In [ ]:
X_test_encoding = encode(X_test_pos)
X_test_encoding[:5]

array([[ 162,   11,  733,   71,  229,  194, 1730, 2570,  733, 1859,  316,
         827,   75, 1928,  829,  606,  121,  325,  469,  125,   71,  229,
        1742,    9, 5199,  419, 1045],
       [ 612,   58,   52, 6273,    6,   49,  606,  113,  743, 1976,  100,
         428,  231,  307, 3318,  368,   39,  428,    4, 2744,  220,  659,
        3687,  908,   75,  262,  796],
       [  43,   26,   56,   59,    4,   71,   16,   17,    1,   98,  186,
          34,  703,   10,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [  43,   26, 1270,    7,   68,   37,   31,  823,  751,  243, 7002,
          36,   10,  773,   37,  666,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [   2,  452,  103,   32,    3,   24,    7,  399,   25,   86,    6,
        4014,  268,  596,    8,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]], dtype=int32)

In [ ]:
predictions = model.predict(X_test_encoding)
predictions

222/222 [==============================] - 3s 10ms/step


array([[5.0191474e-03, 8.9488132e-04, 3.5352593e-03, 2.1780569e-03,
        9.8837262e-01],
       [9.5323008e-03, 7.1391049e-03, 3.6358051e-02, 6.5576397e-03,
        9.4041282e-01],
       [1.0085411e-04, 5.1325903e-04, 1.7060481e-03, 9.9600166e-01,
        1.6781813e-03],
       ...,
       [1.4448462e-03, 1.7584613e-04, 9.9682784e-01, 3.2709294e-04,
        1.2242852e-03],
       [1.0226021e-04, 4.9181143e-04, 1.7986688e-03, 9.9572933e-01,
        1.8778789e-03],
       [1.2980456e-03, 9.8138982e-01, 3.6078857e-03, 1.1043257e-02,
        2.6611013e-03]], dtype=float32)

In [ ]:
def logits_to_onehot_aspects(logits, threshold):
    sig_probabilities = tf.keras.activations.sigmoid(logits).numpy()
    return list(np.where(sig_probabilities > threshold, 1, 0))

# 0.5로 하면 모든 aspect가 1이 됨
aspect_bools = logits_to_onehot_aspects(predictions, 0.6)
pred_series = pd.Series(aspect_bools)
pred_series.head()

0    [0, 0, 0, 0, 1]
1    [0, 0, 0, 0, 1]
2    [0, 0, 0, 1, 0]
3    [0, 0, 0, 1, 0]
4    [1, 0, 0, 0, 0]
dtype: object

In [ ]:
onehot_df["PredList"] = pred_series
onehot_df.head(2)

,RawText,LabelList,PredList
0,(12/25)생각 이상으로 아주 좋습니다. 한번 세탁해서 착용하라는 스티커대로 세탁...,"[1, 0, 0, 1, 0]","[0, 0, 0, 0, 1]"
1,*****최악최악최악최악최악***** 엄마가 선물 받아 기분 좋다고 바로 신고 나가...,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]"


In [ ]:
test_series = onehot_df.apply(lambda x: x["LabelList"] == x["PredList"], axis=1)
test_series.head()

0    [False, True, True, False, False]
1     [True, True, True, False, False]
2     [True, False, True, False, True]
3     [True, False, True, False, True]
4     [False, True, False, True, True]
dtype: object

In [ ]:
# 완전일치
def check_full_accord(x):
    for each in x:
        if not each:
            return 0
    return 1

test_series.apply(check_full_accord).mean()

0.028700692775342852

In [ ]:
# 부분일치
def check_partial_accord(x):
    result = 0
    for each in x:
        if each:
            result += 1
    return result / 5

test_series.apply(check_partial_accord).mean()

0.5603280079174324